# Usman Day 4 HW

# Changing functions to include NumPy

In [120]:
import numpy as np
import math
import random
import time

## Previous functions needed

In [121]:
def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    return atomic_coordinates, box_length

In [122]:
def calculate_tail_correction(num_particles, box_length, cutoff):
    """
    The tail correction associated with using a cutoff radius.
    
    Computes the tail correction based on a cutoff radius used in the LJ energy calculation in reduced units.
    
    Parameters
    ----------
    num_particles : int
        The number of particles in the system.
    
    box_length : int
        Size of the box length of the system, used to calculate volume.
    
    cutoff : int
        Cutoff distance.
    
    Returns
    -------
    tail_correction : float
        The tail correction associated with using the cutoff.
    """
    
    brackets = (1/3*math.pow(1/cutoff,9)) - math.pow(1/cutoff,3)
    volume = box_length**3
    
    constant = ((8*math.pi*(num_particles**2))/(3*volume))
    
    tail_correction = constant * brackets
    
    return tail_correction

In [123]:
def accept_or_reject(delta_U, beta):
    """
    Accept or reject a move based on the Metropolis criterion.
    
    Parameters
    ----------
    detlta_U : float
        The change in energy for moving system from state m to n.
    beta : float
        1/temperature
    
    Returns
    -------
    boolean
        Whether the move is accepted.
    """
    if delta_U <= 0.0:
        accept = True
    else:
        #Generate a random number on (0,1)
        random_number = random.random()
        p_acc = math.exp(-beta*delta_U)
        
        if random_number < p_acc:
            accept = True
        else:
            accept = False
    return accept

In [124]:
def calculate_distance_np(coord1, coord2, box_length=None):
    """
    Calculate the distance between two points. When box_length is set, the minimum image convention is used to calculate the distance between the points.

    Parameters
    ----------
    coord1, coord2 : np.array
        The coordinates of the points, [x, y, z]
    
    box_length : float, optional
        The box length

    Returns
    -------
    distance : float
        The distance between the two points accounting for periodic boundaries
    """
    coord_dist=coord1-coord2
    if box_length:    
            coord_dist = coord_dist - (box_length * np.round(coord_dist/box_length))
    if coord_dist.ndim <2:
        #reshaping the array, make it have 2 dimensions
        coord_dist = coord_dist.reshape (1,-1)
    coord_dist=coord_dist**2
    dist_sum= coord_dist.sum(axis=1)
    distance = np.sqrt(dist_sum)
    return distance

## Creating the required functions using np

In [125]:
def calculate_LJ_np(r_ij):
    """
    The LJ interaction energy between two particles.

    Computes the pairwise Lennard Jones interaction energy based on the separation distance in reduced units.

    Parameters
    ----------
    r_ij : np.array()
        The distance between the particles in reduced units.
    
    Returns
    -------
    pairwise_energy : float
        The pairwise Lennard Jones interaction energy in reduced units.

    Examples
    --------
    >>> calculate_LJ(1)
    0

    """
    #we get a list of r_ij from the distance function in the form of an np.array()
    
    r6_term = (1./r_ij) ** 6
    r12_term = (1./r_ij) ** 12
    pairwise_energy = (r12_term - r6_term)*4
    return pairwise_energy.sum() 


In [126]:
def calculate_pair_energy_np(coordinates, i_particle, box_length, cutoff):
    """
    Calculate the interaction energy of a particle with its environment (all other particles in the system)
    
    Parameters
    ----------
    coordinates : list
        The coordinates for all the particles in the system.
        
    i_particle : int
        The particle number for which to calculate the energy.
        
    cutoff : float
        The simulation cutoff. Beyond this distance, interactions are not calculated.
    
    box_length : float
        The length of the box for periodic bounds
        
    Returns
    -------
    e_total : float
        The pairwise interaction energy of the ith particles with all other particles in the system
    """

    #creates a list of the coordinates for the i_particle
    i_position = coordinates[i_particle]
    
    num_atoms = len(coordinates)
    #i_position has the coordinates for the 1st particle and now we want to see the energy between it and all of its particles
    #that are <cutoff away.
    
    i_position_np = np.array(i_position)
    #new_coords = [x for x in coordinates if x != i_position]
    coordinates_np = np.array(coordinates)
    
    rij = calculate_distance_np(i_position_np, coordinates_np, box_length)
        #should return a list of distances from particle 1 to particle n
        #Now check and see which values are < cutoff and not 0 (distances = 0 are the molecule with itself)
    less_than_cutoff_values = rij[rij<cutoff]
    less_than_cutoff_values3 = less_than_cutoff_values[less_than_cutoff_values != 0]
        #now send all these values to calculate_LJ_np
    e_total = calculate_LJ_np(less_than_cutoff_values3)

    return e_total


In [127]:
def calculate_total_energy_np(coordinates, box_length, cutoff):
    """
    Calculate the total energy of a set of particles using the Lennard Jones potential.
    
    Parameters
    ----------
    coordinates : list
        A nested list containing the x, y,z coordinate for each particle
    box_length : float
        The length of the box. Assumes cubic box.
    cutoff : float
        The cutoff length
    
    Returns
    -------
    total_energy : float
        The total energy of the set of coordinates.
    """
    #similar to calculate_pair_energy_np however in this version we want the TOTAL energy, not just 1 molecule to all others
    #and so we want all molecules to all others.
    total_energy = 0
    num_atoms = len(coordinates)
    #coordinates_np = np.array(coordinates)
    for i in range(num_atoms):
            
            #The commented lines are unneccesary since we are sending the coordinates of one molecule + all the other molecules
            #thus we can just call the calculate_pair_energy_np function which does the same thing
            
            #mol_position = coordinates[i]
            #mol_position_np = np.array(mol_position)
            
            # Calculate the distance between the particles
            
            #dist_ij = calculate_distance_np(coordinates_np, mol_position_np, box_length)
            #less_than_cutoff_values2 = dist_ij[dist_ij<cutoff]
            #less_than_cutoff_values4 = less_than_cutoff_values2[less_than_cutoff_values2 != 0]
            
            # Add to total energy.
            
            #total_energy += calculate_LJ_np(less_than_cutoff_values4)
            total_energy += calculate_pair_energy_np(coordinates, i, box_length, cutoff)
            
            #now divide total_energy by 2 because we counted the energy for every atom twice
    return total_energy/2

## Running the MC simulation using the np functions

In [118]:
start_time = time.time()
# Read or generate initial coordinates
coordinates, box_length = read_xyz('../../lj_sample_configurations/lj_sample_config_periodic1.txt')

# Set simulation parameters
reduced_temperature = 0.9
num_steps = 50000
max_displacement = 0.1
cutoff = 3
#how often to print an update
freq = 1000

# Calculated quantities
beta = 1 / reduced_temperature
num_particles = len(coordinates)

# Energy calculations
total_energy = calculate_total_energy_np(coordinates, box_length, cutoff)
print(total_energy)
total_correction = calculate_tail_correction(num_particles, box_length, cutoff)
print(total_correction)
total_energy += total_correction


for step in range(num_steps):
    # 1. Randomly pick one of the particles.
    random_particle = random.randrange(num_particles)
    # 2. Calculate the interaction energy of the selected particle with the system.
    current_energy = calculate_pair_energy_np(coordinates, random_particle, box_length, cutoff)
    
    # 3. Generate a random x, y, z displacement.
    x_rand = random.uniform(-max_displacement, max_displacement)
    y_rand = random.uniform(-max_displacement, max_displacement)
    z_rand = random.uniform(-max_displacement, max_displacement)
    
    # 4. Modify the coordinate of Nth particle by generated displacements.
    coordinates[random_particle][0] += x_rand
    coordinates[random_particle][1] += y_rand
    coordinates[random_particle][2] += z_rand
    
    # 5. Calculate the interaction energy of the moved particle with the system and store this value.
    proposed_energy = calculate_pair_energy_np(coordinates, random_particle, box_length, cutoff)
    delta_energy = proposed_energy - current_energy
    
    # 6. Calculate if we accept the move based on energy difference.
    accept = accept_or_reject(delta_energy, beta)
    
    # 7. If accepted, move the particle.
    if accept:
        total_energy += delta_energy
    else:
        #Move not accepted, roll back coordinates
        coordinates[random_particle][0] -= x_rand
        coordinates[random_particle][1] -= y_rand
        coordinates[random_particle][2] -= z_rand
    
    # 8. Print the energy if step is a multiple of freq.
    if step % freq == 0:
        print(step, total_energy/num_particles)
        
end_time = time.time()
elapsed_time = end_time-start_time
print(elapsed_time)

-4351.540194543873
-198.4888837441566
0 -5.687536347860037
1000 -5.685708697086865
2000 -5.6657100645464595
3000 -5.6963838279717365
4000 -5.681835027600463
5000 -5.656434890167255
6000 -5.634464339798047
7000 -5.62506546541754
8000 -5.62626620116381
9000 -5.572573326276103
10000 -5.618615942594388
11000 -5.611734149815796
12000 -5.617313989711506
13000 -5.612127559311987
14000 -5.633604832489924
15000 -5.637216894814267
16000 -5.66211999187461
17000 -5.640995410885138
18000 -5.625625391029277
19000 -5.623476291701788
20000 -5.599401885562125
21000 -5.609960999797468
22000 -5.607012235251717
23000 -5.615283862023128
24000 -5.636193126465895
25000 -5.660382475791303
26000 -5.648855033953968
27000 -5.634950114157238
28000 -5.647962620547385
29000 -5.615833446490541
30000 -5.586226490941072
31000 -5.625609013053288
32000 -5.638730277169519
33000 -5.620641659341066
34000 -5.634360049341118
35000 -5.6340216816159625
36000 -5.611794434372244
37000 -5.635718008361222
38000 -5.629175481651825


## Using the Numpy functions are much faster, almost ~90 seconds faster than the functions using the default python functions using the standard python library.